In [ ]:
import pandas as pd
from datetime import time


def calculate_distance_matrix(df)->pd.DataFrame():
    df = pd.read_csv(file_path)
    unique_ids = pd.unique(df[['From', 'To']].values.ravel('K'))
    
    distance_matrix = pd.DataFrame(float('inf'), index=unique_ids, columns=unique_ids)
    
    for id in unique_ids:
        distance_matrix.at[id, id] =0
    for _, row in df.iterrows():
        from_id = row['From']
        to_id = row['To']
        distance = row['Distance']
        
        distance_matrix.at[from_id, to_id] = min(distance_matrix.at[from_id, to_id], distance)
        distance_matrix.at[to_id, from_id] = min(distance_matrix.at[to_id, from_id], distance)

    
    for k in unique_ids:
        for i in unique_ids:
            for j in unique_ids:
                if distance_matrix.at[i, j] > distance_matrix.at[i, k] + distance_matrix.at[k, j]:
                    distance_matrix.at[i, j] = distance_matrix.at[i, k] + distance_matrix.at[k, j]

    return distance_matrix



In [ ]:
def unroll_distance_matrix(df)->pd.DataFrame():

    unrolled_data = []
    
    ids = df.index
    
    for id_start in ids:
        for id_end in ids:
            if id_start != id_end:
                unrolled_data.append([id_start, id_end, df.loc[id_start, id_end]])
    
    df = pd.DataFrame(unrolled_data, columns=['id_start', 'id_end', 'distance'])
 return df


In [ ]:
def find_ids_within_ten_percentage_threshold(df, reference_id)->pd.DataFrame():

    reference_distances = df[df['id_start'] == reference_id]['distance']
    reference_avg = reference_distances.mean()
    
    lower_bound = reference_avg * 0.90
    upper_bound = reference_avg * 1.10
    
    matching_ids = []
    
    for id_start in df['id_start'].unique():
        avg_distance = df[df['id_start'] == id_start]['distance'].mean()
        
        if lower_bound <= avg_distance <= upper_bound:
            matching_ids.append(id_start)
    
    return sorted(matching_ids)





    

In [ ]:
def calculate_toll_rate(df)->pd.DataFrame():
    rate_coefficients = {
        'moto': 0.8,
        'car': 1.2,
        'rv': 1.5,
        'bus': 2.2,
        'truck': 3.6
    }
    
    df['moto'] = df['distance'] * rate_coefficients['moto']
    df['car'] = df['distance'] * rate_coefficients['car']
    df['rv'] = df['distance'] * rate_coefficients['rv']
    df['bus'] = df['distance'] * rate_coefficients['bus']
    df['truck'] = df['distance'] * rate_coefficients['truck']
    
    return df



In [ ]:

def calculate_time_based_toll_rates(df)->pd.DataFrame():
    time_ranges = [
        (time(0, 0), time(10, 0), 0.8),   # 00:00 to 10:00 -> 0.8 factor
        (time(10, 0), time(18, 0), 1.2),  # 10:00 to 18:00 -> 1.2 factor
        (time(18, 0), time(23, 59, 59), 0.8) # 18:00 to 23:59 -> 0.8 factor
    ]
    days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    
    toll_data = []

    for _, row in df.iterrows():
        id_start = row['id_start']
        id_end = row['id_end']
        distance = row['distance']
        moto = row['moto']
        car = row['car']
        rv = row['rv']
        bus = row['bus']
        truck = row['truck']
        
        for day in days_of_week:
            if day in ['Saturday', 'Sunday']:
                discount_factor = 0.7
                start_time = time(0, 0)
                end_time = time(23, 59, 59)
                toll_data.append({
                    'id_start': id_start,
                    'id_end': id_end,
                    'distance': distance,
                    'start_day': day,
                    'end_day': day,
                    'start_time': start_time,
                    'end_time': end_time,
                    'moto': moto * discount_factor,
                    'car': car * discount_factor,
                    'rv': rv * discount_factor,
                    'bus': bus * discount_factor,
                    'truck': truck * discount_factor
                })
            else:
                for start_time, end_time, discount_factor in time_ranges:
                    toll_data.append({
                        'id_start': id_start,
                        'id_end': id_end,
                        'distance': distance,
                        'start_day': day,
                        'end_day': day,
                        'start_time': start_time,
                        'end_time': end_time,
                        'moto': moto * discount_factor,
                        'car': car * discount_factor,
                        'rv': rv * discount_factor,
                        'bus': bus * discount_factor,
                        'truck': truck * discount_factor
                    })
    
    toll_df = pd.DataFrame(toll_data)
    
    return toll_df

